In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
import numpy as np
import pandas as pd
import random
from datetime import datetime
import re

In [7]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext

In [169]:
#population size (MUST BE multiple of workers)
NP=20
#Dimensions
D=3
#bounds
bound = 5
#workers = partitions = islands
workers = 4
#number of itaration for each island evolution
numIteration=2
#mutation scaling Factor
F = 0.6
#CRossover constant
CR = 0.5
#Mutation Strategy
MuSt = "differenceVector"
#Crossover strategy
CRSt = "binomial"

In [170]:
def mutation(a,b,c):
    if(MuSt == "differenceVector"):
        return a+F*(b-c)
    return a+b+c

In [171]:
def crossover(target,donor):
    if(CRSt == "exponential"):  
        n = int(random.random()*D)
        l=1
        while(random.random()<=CR and l<D):
            l=l+1
        for i in range(0,len(target)):
            print("i: ",i)
            for k in range(n,n+l):  
                if(i==k%D):
                    target[i]=donor[i] 
    elif(CRSt == "binomial"): 
        #jrand ensure at least one component is changed
        jrand = int(random.random()*D)
        for j in range(0,len(target)):
            if(random.random()<=CR or j==jrand):
                target[j]=donor[j]
    return target

In [172]:
# IMPO this function will go in another file
# now is here because is easier without the import
def islandsMap(index, iterator): 
    
    oldPop = list(iterator)
    localItaration=0
    #stopping variable
    stop=False
    while(not stop):
        newPop = []
        localItaration=localItaration+1
        #for each crhomosome in the population
        for i in range(0,len(oldPop)):
            targetPartition = oldPop[i][0]
            target = oldPop[i][1]
            #select 2 chromosomes at random
            indA = int(random.random()*len(oldPop))
            while(indA==i):
                indA = int(random.random()*len(oldPop))
            indB = int(random.random()*len(oldPop))
            while(indB==i or indB==indA):
                indB = int(random.random()*len(oldPop))
            indC = int(random.random()*len(oldPop))
            while(indC==i or indC==indB or indC==indA):
                indC = int(random.random()*len(oldPop))
            donor = mutation(oldPop[indA][1],oldPop[indB][1],oldPop[indC][1])
            trial = crossover(target,donor)
            # evaluate the new chromosome and if it is  better substitute it (FOR NOW IS RANDOM)
            if(random.random()>0.9):
                newPop.append((targetPartition,trial))
            else:
                newPop.append((targetPartition,target))
                
        #check stopping criteria
        if(localItaration==numIteration):
            stop=True
        #substitute the population
        oldPop=newPop
            
    return iter(newPop)
    

In [173]:
#create population

#create the NP x D np.matrix with random numbers between +- bound
initialPop = np.random.random_sample((NP,D))*bound*2-bound

#create the NP x D dataframe with random numbers between +- bound
#df2 = pd.DataFrame(np.random.random_sample((NP,D))*bound*2-bound)

#partition the population
rdd = sc.parallelize(initialPop)
#create ann rdd with the partition number for each chromosome at random but evenly distributed
indexes = []
for i in range(0,workers):
    for k in range(0,int(NP/workers)):
        indexes.append(i)
random.shuffle(indexes)
# print("random indexes genereted: " , x)
index = sc.parallelize(indexes)
rdd = index.zip(rdd)
#partition by the key previously computed at random
rdd=rdd.partitionBy(workers)
print("number of partitions: ",rdd.getNumPartitions())
print("Partitions structure: {}".format(rdd.glom().collect()))
rdd.mapPartitionsWithIndex(islandsMap).collect()





number of partitions:  4
Partitions structure: [[(0, array([ 4.34847813,  3.78068006,  2.7948319 ])), (0, array([-4.5111294 ,  2.88548736, -4.05138449])), (0, array([ 3.94437617, -0.69918321, -2.16134014])), (0, array([-0.58595355,  0.3132661 , -1.57275623])), (0, array([ 0.77257177,  0.84187709, -3.6908342 ]))], [(1, array([ 0.01907109,  2.77099133, -3.76259192])), (1, array([ 3.39749082, -0.30037354, -4.87594619])), (1, array([ 0.12683512, -4.8118533 , -4.66869741])), (1, array([-2.54275198,  3.72296714,  2.04076396])), (1, array([-2.71551918,  1.14288615,  1.51107412]))], [(2, array([ 3.55301601, -0.96582483,  1.09701266])), (2, array([-2.77807917,  0.10809944,  3.05873371])), (2, array([-0.9814001 , -2.48377252, -2.7045901 ])), (2, array([-3.44085264,  2.48356198,  3.75147239])), (2, array([ 4.32348209, -1.24629528, -0.49329517]))], [(3, array([-1.47054911, -4.34547485,  4.99702844])), (3, array([ 1.33357503, -2.46645338, -4.88078651])), (3, array([-0.04886125, -3.6822795 , -0.8848

[(0, array([-5.6592569 , -0.35308217, -0.67416319])),
 (0, array([-8.37745473, -1.93939261, -4.05138449])),
 (0, array([-5.33347273, -1.82521264, -5.32223127])),
 (0, array([-11.14927279,  -3.11953882,  -1.57275623])),
 (0, array([-12.78019149,  -4.07132509,  -3.59908901])),
 (1, array([-0.58035856,  0.28103449, -0.51867131])),
 (1, array([-2.18211082, -2.68725805, -4.87594619])),
 (1, array([-3.35954688, -3.24455555, -5.1937601 ])),
 (1, array([-0.26031355, -3.35363859,  2.04076396])),
 (1, array([-1.22136491, -5.13461412, -0.57360097])),
 (2, array([ 3.55301601,  2.11812644, -0.57337277])),
 (2, array([-5.95864535,  6.80929251,  3.64668866])),
 (2, array([-10.15496654,   7.02855384,   4.46197346])),
 (2, array([-14.18343488,   9.75554895,   0.78543522])),
 (2, array([-1.38185771,  3.8858803 ,  2.06416562])),
 (3, array([-1.47054911,  0.02770177, -2.16149171])),
 (3, array([ -5.0809788 ,  10.39601987,  -5.55910934])),
 (3, array([ -6.67381542,  12.34637189,  -6.37361133])),
 (3, array